In [1]:
import apache_beam   # Needs to be imported separately to avoid TypingError
import weatherbench2
import xarray as xr
from weatherbench2 import config
import numpy as np
import matplotlib.pyplot as plt


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
forecast_path = 'gs://weatherbench2/datasets/hres/2016-2022-0012-64x32_equiangular_with_poles_conservative.zarr'
obs_path = 'gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_with_poles_conservative.zarr'
climatology_path = 'gs://weatherbench2/datasets/era5-hourly-climatology/1990-2019_6h_64x32_equiangular_with_poles_conservative.zarr'

In [3]:
observations = xr.open_zarr(obs_path)


In [4]:
climatology = xr.open_zarr(climatology_path)
climatology

<xarray.Dataset>
Dimensions:                                      (hour: 4, dayofyear: 366,
                                                  longitude: 64, latitude: 32,
                                                  level: 13)
Coordinates:
  * dayofyear                                    (dayofyear) int64 1 2 ... 366
  * hour                                         (hour) int64 0 6 12 18
  * latitude                                     (latitude) float64 -90.0 ......
  * level                                        (level) int64 50 100 ... 1000
  * longitude                                    (longitude) float64 0.0 ... ...
Data variables: (12/28)
    10m_u_component_of_wind                      (hour, dayofyear, longitude, latitude) float32 dask.array<chunksize=(4, 366, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind                      (hour, dayofyear, longitude, latitude) float32 dask.array<chunksize=(4, 366, 64, 32), meta=np.ndarray>
    10m_wind_speed                               (hour, dayofyear, longitude, latitude) float32 dask.array<chunksize=(4, 366, 64, 32), meta=np.ndarray>
    2m_temperature                               (hour, dayofyear, longitude, latitude) float32 dask.array<chunksize=(4, 366, 64, 32), meta=np.ndarray>
    geopotential                                 (hour, dayofyear, level, longitude, latitude) float32 dask.array<chunksize=(4, 366, 13, 64, 32), meta=np.ndarray>
    mean_sea_level_pressure                      (hour, dayofyear, longitude, latitude) float32 dask.array<chunksize=(4, 366, 64, 32), meta=np.ndarray>
    ...                                           ...
    total_precipitation_6hr_seeps_dry_fraction   (hour, dayofyear, longitude, latitude) float32 dask.array<chunksize=(4, 366, 64, 32), meta=np.ndarray>
    total_precipitation_6hr_seeps_threshold      (hour, dayofyear, longitude, latitude) float32 dask.array<chunksize=(4, 366, 64, 32), meta=np.ndarray>
    u_component_of_wind                          (hour, dayofyear, level, longitude, latitude) float32 dask.array<chunksize=(4, 366, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind                          (hour, dayofyear, level, longitude, latitude) float32 dask.array<chunksize=(4, 366, 13, 64, 32), meta=np.ndarray>
    vertical_velocity                            (hour, dayofyear, level, longitude, latitude) float32 dask.array<chunksize=(4, 366, 13, 64, 32), meta=np.ndarray>
    wind_speed                                   (hour, dayofyear, level, longitude, latitude) float32 dask.array<chunksize=(4, 366, 13, 64, 32), meta=np.ndarray>

In [5]:
climatology.sel(dayofyear=6, level=50, hour=0, latitude=-90.0, longitude=0.0)["geopotential"].values

In [ ]:
def climatology_model(climatology_data, variable, day_of_year, level=None, hour=None):
    # Extract the climatology data for the variable and, if applicable, the level
    if level:
        variable_climatology = climatology_data[variable].sel(level=level)
    else:
        variable_climatology = climatology_data[variable]

    if hour:
        variable_climatology = variable_climatology.sel(hour=hour)
    
    # Return the climatological average for the given day of the year and hour across all longtitude and latitude
    climatology_average = variable_climatology.sel(dayofyear=day_of_year).mean()

    return climatology_average

In [ ]:
climatology_model(climatology, "geopotential", day_of_year=i, level=50).values

array(199406.53, dtype=float32)

In [ ]:
# plot climatology model 
y = []
x = []
for i in range(1, 366):
    y.append(climatology_model(climatology, "geopotential", day_of_year=i, level=50).values)
    x.append(i)

In [ ]:
# Create the time plot
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
plt.plot(x, y, marker='o', linestyle='-')  # 'o' for markers, '-' for a solid line
plt.title('Time Plot')
plt.xlabel('Time')
plt.ylabel('Values')
plt.grid(True)  # Add a grid


In [ ]:
observations

<xarray.Dataset>
Dimensions:                                           (time: 92040,
                                                       longitude: 64,
                                                       latitude: 32, level: 13)
Coordinates:
  * latitude                                          (latitude) float64 -90....
  * level                                             (level) int64 50 ... 1000
  * longitude                                         (longitude) float64 0.0...
  * time                                              (time) datetime64[ns] 1...
Data variables: (12/38)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind                           (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    10m_wind_speed                                    (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    2m_temperature                                    (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  (longitude, latitude) float32 dask.array<chunksize=(64, 32), meta=np.ndarray>
    anisotropy_of_sub_gridscale_orography             (longitude, latitude) float32 dask.array<chunksize=(64, 32), meta=np.ndarray>
    ...                                                ...
    type_of_high_vegetation                           (longitude, latitude) float32 dask.array<chunksize=(64, 32), meta=np.ndarray>
    type_of_low_vegetation                            (longitude, latitude) float32 dask.array<chunksize=(64, 32), meta=np.ndarray>
    u_component_of_wind                               (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind                               (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    vertical_velocity                                 (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    wind_speed                                        (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>

In [ ]:
import numpy as np
from scipy.stats import gaussian_kde

def climatology_probabilistic_model(obs_data, variable, day_of_year):
    """
    Create a probabilistic climatology model based on daily means for a specific year.

    Parameters:
    - obs_data (xarray.Dataset): Observed data containing time, longitude, latitude, level, and the chosen climate variable.
    - variable (str): The name of the climate variable to model (e.g., "temperature").
    - year (int): The year for which to calculate the climatology.

    Returns:
    - pdf (gaussian_kde): The estimated probability distribution function.
    """
    years_list = []
    for year in range(1959, 2022):
        years_list.append(year)
    predictions = np.array([])

    # Create empty arrays with these coordinates
    latitude_arr = xr.DataArray(obs_data.latitude.values, dims='latitude')
    level_arr = xr.DataArray(obs_data.level.values, dims='level')
    longitude_arr = xr.DataArray(obs_data.longitude.values, dims='longitude')
    dayofyear_arr = xr.DataArray(np.array([day_of_year]), dims='dayofyear')

    # Create an empty dataset with these dimensions
    forecast_probabilities = xr.Dataset(
        {'latitude': latitude_arr, 'level': level_arr, 'longitude': longitude_arr, 'dayofyear': dayofyear_arr}
    )

    for i in years_list:
        # Filter data for the chosen year
        obs_data_year = obs_data.sel(time=obs_data.time.dt.year == i)
        # Calculate daily means
        daily_means = obs_data_year.groupby('time.dayofyear').mean(dim='time')[variable]
        forecast = daily_means.sel(dayofyear = day_of_year)
        return forecast
        forecast_probabilities = xr.concat([forecast_probabilities, forecast], dim='latitude')

        
    return forecast_probabilities


In [ ]:
climate = climatology_probabilistic_model(observations, variable="geopotential", day_of_year=2).sel(dayofyear = 2)

In [ ]:
climate

<xarray.DataArray 'geopotential' (level: 13, longitude: 64, latitude: 32)>
dask.array<getitem, shape=(13, 64, 32), dtype=float32, chunksize=(13, 64, 32), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 -90.0 -84.19 -78.39 ... 78.39 84.19 90.0
  * level      (level) int64 50 100 150 200 250 300 400 500 600 700 850 925 1000
  * longitude  (longitude) float64 0.0 5.625 11.25 16.88 ... 343.1 348.8 354.4
    dayofyear  int64 2
Attributes:
    long_name:      Geopotential
    short_name:     z
    standard_name:  geopotential
    units:          m**2 s**-2

In [ ]:
climate2 = climatology_probabilistic_model(observations, variable="geopotential", day_of_year=2)

In [ ]:
climate2

<xarray.DataArray 'geopotential' (level: 13, longitude: 64, latitude: 32)>
dask.array<getitem, shape=(13, 64, 32), dtype=float32, chunksize=(13, 64, 32), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 -90.0 -84.19 -78.39 ... 78.39 84.19 90.0
  * level      (level) int64 50 100 150 200 250 300 400 500 600 700 850 925 1000
  * longitude  (longitude) float64 0.0 5.625 11.25 16.88 ... 343.1 348.8 354.4
    dayofyear  int64 2
Attributes:
    long_name:      Geopotential
    short_name:     z
    standard_name:  geopotential
    units:          m**2 s**-2

In [ ]:
s